## Process and Run Experiment on the Helmet Dataset

This notebook contains all code needed to train use Peoplenet to inference a dataset and then train with an added class

This notebook requires the TAO Launcher, Docker and NGC to be setup

The notebook also requires a helmet dataset from kaggle which can be found here https://www.kaggle.com/andrewmvd/helmet-detection

It must be downloaded and the archive.zip placed in ../../datasets/helmet

This notebook takes the following steps

1) Clear past results  
2) Convert the dataset into kitti format  
3) Download PeopleNet model  
4) Map local directories the the TAO launcher  
5) Inference the images with PeopleNet and combine labels  
6) Generate TF Records  
7) Train PeopleNet with the helmet class  
8) Graph Results  

### Clear past results

In [ ]:
#run this cell to clear past results
#May need to run the following command outside the noteobok: sudo chown -R $USER /path/to/github/repo
!rm -rf $dataset_home/helmet_set_all 
!rm -rf $dataset_home/annotations 
!rm -rf $dataset_home/images 
!rm -rf $dataset_home/peoplenet_labels 
!rm -rf $exp_home/trained_inf_out 
!rm -rf $exp_home/train_out 

### Convert the dataset into kitti format

In [ ]:
!python3 -m pip install matplotlib

In [ ]:
import os
from preprocess_helmet import *

In [ ]:
#local file paths
repo_home = os.path.join(os.getcwd(), "../../")
dataset_home = os.path.join(repo_home, "datasets/helmet")
exp_home = os.path.join(os.getcwd(), "experiments")
workspace_home = os.path.join(repo_home, "workspace")
models_home = os.path.join(repo_home, "workspace/models")

#file paths inside the container
dataset_home_cont = "/datasets/helmet"
exp_home_cont = "/tlt_exp/peoplenet_helmet/experiments"

In [ ]:
%cd $dataset_home
!unzip $dataset_home/archive.zip

In [ ]:
#Setup output paths for labels and images
label_out = os.path.join(dataset_home, "helmet_set_all/labels")
image_out = os.path.join(dataset_home, "helmet_set_all/images")

os.makedirs(image_out, exist_ok=True)
os.makedirs(label_out, exist_ok=True)

#move all the images to the right folder
!mv $dataset_home/images/* $image_out

#Convert xml labels to kitti txt file labels and put them in the output path
xml_labels = os.path.join(dataset_home, "annotations")
for label in os.listdir(xml_labels):
    convert_annotation(os.path.join(xml_labels,label), label_out)

### Download PeopleNet model

In [ ]:
os.makedirs(os.path.join(models_home, "detectnet_v2"), exist_ok=True)
%cd $models_home/detectnet_v2
!ngc registry model download-version "nvidia/tlt_peoplenet:unpruned_v2.1"

## Map local directories the the TAO launcher

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.path.join(repo_home, "datasets"),
            "destination": "/datasets"
        },
        # Mapping the specs directory.
        {
            "source": os.path.join(repo_home, "workspace"),
            "destination": "/tlt_exp"
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
!cat ~/.tlt_mounts.json

### Inference the images with PeopleNet and combine labels

In [ ]:
#convert to jpg
#from PIL import Image
#image_folder = os.path.join(dataset_home, "helmet_set_all/images")
#for im_path in os.listdir(image_folder):
#    im = Image.open(os.path.join(image_folder, im_path))
#    im = im.convert("RGB")
#    im.save(os.path.join(image_folder, im_path.split(".")[0] + ".jpg"))
#
#!rm $image_folder/*.png

In [ ]:
#Setup variables to inference the images with peoplenet
inference_spec = os.path.join(exp_home_cont,"inf_people_spec.txt")
output_folder = os.path.join(dataset_home_cont, "peoplenet_labels")
dataset_images = os.path.join(dataset_home_cont, "helmet_set_all/images")
key = "tlt_encode"

#inference the helmet dataset images with peoplenet
!tao detectnet_v2 inference -e $inference_spec -o $output_folder -i $dataset_images -k $key

In [ ]:
#Combine the inferenced labels with the existing helmet labels
output_folder = os.path.join(dataset_home, "peoplenet_labels")

#loop through all labels in helmet dataset
for label in os.listdir(os.path.join(dataset_home,"helmet_set_all/labels")):
    helmet_label = os.path.join(dataset_home, "helmet_set_all/labels", label)
    inferenced_labels = os.path.join(output_folder, "labels")
    people_label = os.path.join(output_folder,"labels",label)
    #append the peoplenet inference to the original helmet labels
    with open(helmet_label, "a") as label_f:
        with open(people_label, "r") as people_f:
            for line in people_f:
                line = line.split(" ")
                line = " ".join(line[:-1])
                label_f.write(line + "\n")
    #output is a label text file with the helmet, people and face labels

### Generate TF Records

In [ ]:
 def gen_tf_spec(dataset_path):

    spec_str = f"""
    kitti_config {{
      root_directory_path: "{dataset_path}"
      image_dir_name: "images"
      label_dir_name: "labels"
      image_extension: ".png"
      partition_mode: "random"
      num_partitions: 2
      val_split: 20
      num_shards: 10
    }}
    """
    return spec_str

In [ ]:
#setup paths for tf record generation
path = os.path.join(dataset_home, "helmet_set_all")
path_cont = os.path.join(dataset_home_cont, "helmet_set_all") #path inside container

record_path = os.path.join(path, "tfrecord_spec.txt")

#Write the tf record spec file
with open(record_path, "w+") as spec:
    spec.write(gen_tf_spec(path_cont))
    
record_output = os.path.join(path_cont, "tfrecords/") 
record_path = os.path.join(path_cont, "tfrecord_spec.txt")

#Generate the tf records
!tao detectnet_v2 dataset_convert -d $record_path -o $record_output

### Train PeopleNet with the helmet class

In [ ]:
#Setup paths for training and inference
train_out = os.path.join(exp_home, "train_out")
train_out_cont = os.path.join(exp_home_cont, "train_out")
train_spec_path = os.path.join(exp_home_cont, "train_spec.txt")
inf_spec_path = os.path.join(exp_home_cont, "inf_new_spec.txt")
model_out = os.path.join(exp_home_cont, "train_out")
trained_model = os.path.join(model_out, "final_model.tlt")

In [ ]:
!tao detectnet_v2 train -e $train_spec_path -r $train_out_cont -n "final_model" -k "tlt_encode"


In [ ]:
!tao detectnet_v2 inference -e $inf_spec_path -i "/datasets/helmet/helmet_set_all/images" -o "/tlt_exp/peoplenet_helmet/experiments/trained_inf_out" -k "tlt_encode"

### Graph Results

In [ ]:
import matplotlib.pyplot as plt   
def get_map_data(filepath):
    x_vals_map = []
    y_vals_map = []
    with open(filepath, "r") as f:
        epoch = 0
        for line in f:
            data = eval(line)
            if "cur_epoch" in data.keys():
                epoch = data["cur_epoch"]

            elif "mean average precision" in data.keys():
                mAP = data["average_precision"]["withhelmet"]
                y_vals_map.append(mAP)
                x_vals_map.append(epoch)

    return x_vals_map, y_vals_map

In [ ]:
status_file = os.path.join(train_out, "status.json")

plt.figure(figsize=(10,5))
plt.title('Helmet AP Over Epoch')
plt.xlabel("Epoch")
plt.ylabel("AP %")
plt.ylim([0,100])
plt.yticks(range(0,101,10))
plt.tick_params(right=True, labelright=True)

x,y = get_map_data(status_file)
print(status_file + "\n max AP: " + str(max(y)) + " \n")
plt.plot(x,y)
plt.show()